# M05 Homework

```yaml
course:   DS 5001 
topic:    BOW and TFIDF
author:   JiHo Lee (qxz6hb)
```

### Question 1. Show the function you created.

> The codes are below.

### Question 2. What are the top 20 words in the corpus by TFIDF mean using the `max` count method and `book` as the bag?

> elinor       <br>
pierre        <br>
vernon        <br>
marianne      <br>
emma          <br>
darcy         <br>
reginald     <br>
babbalanja    <br>
catherine     <br>
frederica     <br>
crawford      <br>
fanny         <br>
elliot        <br>
weston        <br>
media         <br>
israel        <br>
knightley     <br>
tilney        <br>
elton         <br>
bingley       <br>

### Question 3. What are the top 20 words in the corpus by TFIDF mean, if you using the `sum` count method and `chapter` as the bag? 

> her             <br>
she             <br>
cosmopolitan    <br>
pierre          <br>
communion      <br>
i               <br>
sailors         <br>
you             <br>
hypothetical    <br>
mr              <br>
and             <br>
confidential    <br>
the             <br>
dream           <br>
boon            <br>
mrs             <br>
elephants       <br>
whale           <br>
thou            <br>
acquaintance    <br>

### Question 4. Characterize the general difference between the words in Question 3 and those in Question 2 in terms of part-of-speech.

> The top 20 words in Question 2 were calculated using TF-IDF with max at book level. This means that it is likely to have unique or significant terms that relevant in specific contexts. Therefore, these words mostly consist of nameas, nouns, and terms related to the theme of the book.
<br><br>
The top 20 words in Question 3 wre calculated using TF-IDF with sum at chapter level. Thus, they are likely to include verbs, adjectives, or commonly used terms within the chapter across the books. Thus, the top 20 words generally represent terms frequently used throughout the book. Therefore, you can se e these are consist of her, i, you etc.


### Question 5. Compute mean `TFIDF` for vocabularies conditioned on individual author, using *chapter* as the bag and `max` as the `TF` count method. Among the two authors, whose work has the most significant adjective?

> Jane Austen <br>
_thy_, and _dear_ had the most significant TFIDF values with 0.034276, 0.058711 from individual author, _Herman Melville_ and _Jane Austen_ respectively. Since _dear_ has the most significant value, _Jane Austen_'s work has the most significant adjective.


```The code related to questions and answers with explanation is described in each corresponding section below.```

### Set Up
### Import, config, and import the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly_express as px

sns.set()

import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home'] 
output_dir = config['DEFAULT']['output_dir']
data_prefix = 'austen-melville'


data_home = data_home.replace('/', '\\')
output_dir = output_dir.replace('/', '\\')

OHCO = ['book_id', 'chap_id', 'para_num', 'sent_num', 'token_num']
bags = dict(
    SENTS = OHCO[:4],
    PARAS = OHCO[:3],
    CHAPS = OHCO[:2],
    BOOKS = OHCO[:1]
)

LIB = pd.read_csv(f"{output_dir}\\{data_prefix}-LIB.csv").set_index('book_id')
TOKEN = pd.read_csv(f'{output_dir}\\{data_prefix}-CORPUS.csv').set_index(OHCO).dropna()
# VOCAB = pd.read_csv(f'{output_dir}\\{data_prefix}-VOCAB.csv').set_index('term_str').dropna()
# POS_GROUP = pd.read_csv(f'{output_dir}/{data_prefix}-POS_GROUP.csv').set_index('pos_group')

### <mark>Question 1.</mark>

The function to generate a BOW

In [2]:
def gen_bag(TOKEN, ohco_level):
    
    bag_tmp = ohco_level
    ans = TOKEN.groupby(bags[bag_tmp]+['term_str']).term_str.count().to_frame('n') 
    
    return ans

### <mark>Question 1.</mark>

The function that returns the TFIDF values for a given BOW

In [3]:
def cal_tfidf(BOW, TF_type):
#     log2(N/DF)
#     N : the number of documents (aka 'bags') in the BOW

    DTCM = BOW.n.unstack(fill_value=0)
    method = TF_type
    print('TF method:', method)

    if method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif method == 'raw':
        TF = DTCM.T
    elif method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    
    DF = DTCM.astype('bool').sum() 
    N = DTCM.shape[0]
    
    idf_method = 'standard'   # standard, max, smooth
    if idf_method == 'standard':
        IDF = np.log2(N / DF)
    elif idf_method == 'max':
        IDF = np.log2(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log2((1 + N) / (1 + DF)) + 1
    
    TFIDF = TF * IDF
    
    return TFIDF
# BOW['tf'] = TF.stack()
# BOW['tfidf'] = TFIDF.stack()

### <mark>Question 2.</mark>
Top 20 words and their TFIDF values 

In [4]:
ans = gen_bag(TOKEN, 'BOOKS')
ans.head(10)

ans_TFIDF = cal_tfidf(ans, 'max')

ans_TFIDF.mean(axis=0).sort_values(ascending=False)[:20]
# ans_TFIDF.sort_values('pkr').head(20).style.background_gradient(cmap='YlGnBu')

TF method: max


term_str
elinor        0.033840
pierre        0.030911
vernon        0.025980
marianne      0.021347
emma          0.021164
darcy         0.019302
reginald      0.018486
babbalanja    0.018252
catherine     0.018238
frederica     0.017986
crawford      0.017749
fanny         0.017167
elliot        0.017053
weston        0.016591
media         0.015986
israel        0.015428
knightley     0.015184
tilney        0.013815
elton         0.013648
bingley       0.013264
dtype: float64

### <mark>Question 3.</mark>
Top 20 words and their TFIDF values

In [5]:
ans = gen_bag(TOKEN, 'CHAPS')

ans_TFIDF = cal_tfidf(ans, 'sum')
# ans_TFIDF.head()
ans_TFIDF.mean(axis=0).sort_values(ascending=False)[:20]

TF method: sum


term_str
her             0.004327
she             0.004150
cosmopolitan    0.003485
pierre          0.003317
communion       0.003004
i               0.002771
sailors         0.002668
you             0.002620
hypothetical    0.002437
mr              0.002084
and             0.002054
confidential    0.002042
the             0.001972
dream           0.001942
boon            0.001857
mrs             0.001747
elephants       0.001731
whale           0.001715
thou            0.001696
acquaintance    0.001690
dtype: float64

### <mark>Question 5.</mark>
Jane Austen <br>
_thy_, and _dear_ had the most significant TFIDF values with 0.034276, 0.058711 from individual author, _Herman Melville_ and _Jane Austen_ respectively. Since _dear_ has the most significant value, _Jane Austen_'s work has the most significant adjective.

In [6]:
VOCAB = TOKEN.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = TOKEN[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = TOKEN[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

ans = TOKEN.groupby(bags['CHAPS']+['term_str']).term_str.count().to_frame('n') 
ans = ans.join(LIB[['author', 'title']])
# ans['author'] = ans.set_index('author')

ans_TFIDF = cal_tfidf(ans, 'max')
ans_TFIDF =  ans_TFIDF.join(LIB[['author']], how = 'left', lsuffix='', rsuffix='right')
tmp = ans_TFIDF.groupby('authorright').mean()

tmp.columns.name = 'term_str'
adjectives = VOCAB[VOCAB['max_pos'].isin(['JJ', 'JJR', 'JJS'])].index
tmp_adj = tmp.loc[:, tmp.columns.intersection(adjectives)]
tmp_adj.columns.name = 'term_str'

print(tmp_adj[tmp_adj.index=='MELVILLE, HERMAN'].mean(axis=0).sort_values(ascending=False)[:1])
print()
print(tmp_adj[tmp_adj.index=='AUSTEN, JANE'].mean(axis=0).sort_values(ascending=False)[:1])

TF method: max
term_str
thy    0.034276
dtype: float64

term_str
dear    0.058711
dtype: float64
